# Tutorial 04: computation of the inf-sup constant for a Stokes problem discretization

In this tutorial we compare the computation of the inf-sup constant of a Stokes problem using standard assembly with mixed function spaces and block assembly.

In [ ]:
import typing

In [ ]:
import dolfinx.cpp
import dolfinx.fem
import dolfinx.mesh
import mpi4py
import numpy as np
import numpy.typing as npt
import petsc4py
import slepc4py
import ufl

In [ ]:
import multiphenicsx.fem
import multiphenicsx.io

### Mesh

In [ ]:
mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, 32, 32)


def wall(x: npt.NDArray[np.float64]) -> npt.NDArray[bool]:
    """Determine the position of the wall."""
    return np.logical_or(x[1] < 0 + np.finfo(float).eps, x[1] > 1 - np.finfo(float).eps)


boundary_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, wall)

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

### Function spaces

In [ ]:
V_element = ufl.VectorElement("Lagrange", mesh.ufl_cell(), 2)
Q_element = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

### Auxiliary function for eigenvector normalization

In [ ]:
def normalize(u1: dolfinx.fem.Function, u2: dolfinx.fem.Function, p: dolfinx.fem.Function) -> None:
    """Normalize an eigenvector."""
    u1_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(u1), ufl.grad(u1)) * ufl.dx), op=mpi4py.MPI.SUM))
    u1.vector.scale(1. / u1_norm)
    u1.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    u2_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(u2), ufl.grad(u2)) * ufl.dx), op=mpi4py.MPI.SUM))
    u2.vector.scale(1. / u2_norm)
    u2.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    p_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(p * p * ufl.dx), op=mpi4py.MPI.SUM))
    p.vector.scale(1. / p_norm)
    p.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

### Standard FEniCSx formulation using a mixed function space

In [ ]:
def run_monolithic() -> typing.Tuple[np.float64, dolfinx.fem.Function]:
    """Run standard FEniCSx formulation using a mixed function space."""
    # Function spaces
    W_element = ufl.MixedElement(V_element, Q_element)
    W = dolfinx.fem.FunctionSpace(mesh, W_element)

    # Test and trial functions: monolithic
    vq = ufl.TestFunction(W)
    (v, q) = ufl.split(vq)
    up = ufl.TrialFunction(W)
    (u, p) = ufl.split(up)

    # Variational forms
    lhs = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx - ufl.div(v) * p * ufl.dx - ufl.div(u) * q * ufl.dx
    rhs = - ufl.inner(p, q) * ufl.dx

    # Define restriction for DOFs associated to homogenous Dirichlet boundary conditions
    dofs_W = np.arange(0, W.dofmap.index_map.size_local + W.dofmap.index_map.num_ghosts)
    bdofs_V = dolfinx.fem.locate_dofs_topological(W.sub(0), mesh.topology.dim - 1, boundary_facets)
    bdofs_V = dolfinx.fem.locate_dofs_topological(
        (W.sub(0), W.sub(0).collapse()), mesh.topology.dim - 1, boundary_facets)[0]
    restriction = multiphenicsx.fem.DofMapRestriction(W.dofmap, np.setdiff1d(dofs_W, bdofs_V))

    # Assemble lhs and rhs matrices
    A = multiphenicsx.fem.assemble_matrix(lhs, restriction=(restriction, restriction))
    A.assemble()
    B = multiphenicsx.fem.assemble_matrix(rhs, restriction=(restriction, restriction))
    B.assemble()

    # Solve
    eps = slepc4py.SLEPc.EPS().create(mesh.comm)
    eps.setOperators(A, B)
    eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GNHEP)
    eps.setDimensions(1, petsc4py.PETSc.DECIDE, petsc4py.PETSc.DECIDE)
    eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1.e-5)
    eps.getST().setType(slepc4py.SLEPc.ST.Type.SINVERT)
    eps.getST().getKSP().setType("preonly")
    eps.getST().getKSP().getPC().setType("lu")
    eps.getST().getKSP().getPC().setFactorSolverType("mumps")
    eps.solve()
    assert eps.getConverged() >= 1

    # Extract leading eigenvalue and eigenvector
    vr = dolfinx.cpp.fem.petsc.create_vector_block([(restriction.index_map, restriction.index_map_bs)])
    vi = dolfinx.cpp.fem.petsc.create_vector_block([(restriction.index_map, restriction.index_map_bs)])
    eigv = eps.getEigenpair(0, vr, vi)
    r, i = eigv.real, eigv.imag
    assert abs(i) < 1.e-10
    assert r > 0., "r = " + str(r) + " is not positive"
    print("Inf-sup constant (monolithic): ", np.sqrt(r))

    # Transform eigenvector into eigenfunction
    r_fun = dolfinx.fem.Function(W)
    vr.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    with r_fun.vector.localForm() as r_fun_local, \
            multiphenicsx.fem.VecSubVectorWrapper(vr, W.dofmap, restriction) as vr_wrapper:
        r_fun_local[:] = vr_wrapper
    (u_fun_1, u_fun_2) = (r_fun.sub(0).sub(0).collapse(), r_fun.sub(0).sub(1).collapse())
    p_fun = r_fun.sub(1).collapse()
    normalize(u_fun_1, u_fun_2, p_fun)

    return (r, u_fun_1, u_fun_2, p_fun)

In [ ]:
(eig_m, u_fun_1_m, u_fun_2_m, p_fun_m) = run_monolithic()

In [ ]:
multiphenicsx.io.plot_scalar_field(u_fun_1_m, "u1")

In [ ]:
multiphenicsx.io.plot_scalar_field(u_fun_2_m, "u2")

In [ ]:
multiphenicsx.io.plot_scalar_field(p_fun_m, "p")

### Block FEniCSx formulation using two independent function spaces

In [ ]:
def run_block() -> typing.Tuple[np.float64, dolfinx.fem.Function]:
    """Run block FEniCSx formulation using two independent function spaces."""
    # Function spaces
    V = dolfinx.fem.FunctionSpace(mesh, V_element)
    Q = dolfinx.fem.FunctionSpace(mesh, Q_element)

    # Test and trial functions
    (v, q) = (ufl.TestFunction(V), ufl.TestFunction(Q))
    (u, p) = (ufl.TrialFunction(V), ufl.TrialFunction(Q))

    # Variational forms
    lhs = [[ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx, - ufl.div(v) * p * ufl.dx],
           [- ufl.div(u) * q * ufl.dx, None]]
    rhs = [[None, None],
           [None, - p * q * ufl.dx]]
    rhs[0][0] = dolfinx.fem.Constant(mesh, 0.) * ufl.inner(u, v) * ufl.dx

    # Define restriction for DOFs associated to homogenous Dirichlet boundary conditions
    dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
    bdofs_V = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, boundary_facets)
    dofs_Q = np.arange(0, Q.dofmap.index_map.size_local + Q.dofmap.index_map.num_ghosts)
    restriction_V = multiphenicsx.fem.DofMapRestriction(V.dofmap, np.setdiff1d(dofs_V, bdofs_V))
    restriction_Q = multiphenicsx.fem.DofMapRestriction(Q.dofmap, dofs_Q)
    restriction = [restriction_V, restriction_Q]

    # Assemble lhs and rhs matrices
    A = multiphenicsx.fem.assemble_matrix_block(lhs, bcs=[], restriction=(restriction, restriction))
    A.assemble()
    B = multiphenicsx.fem.assemble_matrix_block(rhs, bcs=[], restriction=(restriction, restriction))
    B.assemble()

    # Solve
    eps = slepc4py.SLEPc.EPS().create(mesh.comm)
    eps.setOperators(A, B)
    eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GNHEP)
    eps.setDimensions(1, petsc4py.PETSc.DECIDE, petsc4py.PETSc.DECIDE)
    eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1.e-5)
    eps.getST().setType(slepc4py.SLEPc.ST.Type.SINVERT)
    eps.getST().getKSP().setType("preonly")
    eps.getST().getKSP().getPC().setType("lu")
    eps.getST().getKSP().getPC().setFactorSolverType("mumps")
    eps.solve()
    assert eps.getConverged() >= 1

    # Extract leading eigenvalue and eigenvector
    vr = dolfinx.cpp.fem.petsc.create_vector_block(
        [(restriction_.index_map, restriction_.index_map_bs) for restriction_ in restriction])
    vi = dolfinx.cpp.fem.petsc.create_vector_block(
        [(restriction_.index_map, restriction_.index_map_bs) for restriction_ in restriction])
    eigv = eps.getEigenpair(0, vr, vi)
    r, i = eigv.real, eigv.imag
    assert abs(i) < 1.e-10
    assert r > 0., "r = " + str(r) + " is not positive"
    print("Inf-sup constant (block): ", np.sqrt(r))

    # Transform eigenvector into eigenfunction
    (u_fun, p_fun) = (dolfinx.fem.Function(V), dolfinx.fem.Function(Q))
    vr.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    with multiphenicsx.fem.BlockVecSubVectorWrapper(vr, [V.dofmap, Q.dofmap], restriction) as vr_wrapper:
        for vr_wrapper_local, component in zip(vr_wrapper, (u_fun, p_fun)):
            with component.vector.localForm() as component_local:
                component_local[:] = vr_wrapper_local
    (u_fun_1, u_fun_2) = (u_fun.sub(0).collapse(), u_fun.sub(1).collapse())
    normalize(u_fun_1, u_fun_2, p_fun)

    return (r, u_fun_1, u_fun_2, p_fun)

In [ ]:
(eig_b, u_fun_1_b, u_fun_2_b, p_fun_b) = run_block()

In [ ]:
multiphenicsx.io.plot_scalar_field(u_fun_1_b, "u1")

In [ ]:
multiphenicsx.io.plot_scalar_field(u_fun_2_b, "u2")

In [ ]:
multiphenicsx.io.plot_scalar_field(p_fun_b, "p")

### Error computation between standard and block formulations

In [ ]:
def run_error(
    eig_m: np.float64, eig_b: np.float64, u_fun_1_m: dolfinx.fem.Function, u_fun_1_b: dolfinx.fem.Function,
    u_fun_2_m: dolfinx.fem.Function, u_fun_2_b: dolfinx.fem.Function, p_fun_m: dolfinx.fem.Function,
    p_fun_b: dolfinx.fem.Function
) -> None:
    """Compute errors between the mixed and block cases."""
    err_inf_sup = np.abs(np.sqrt(eig_b) - np.sqrt(eig_m)) / np.sqrt(eig_m)
    print("Relative error for inf-sup constant equal to", err_inf_sup)
    assert np.isclose(err_inf_sup, 0., atol=1.e-8)
    # Even after normalization, eigenfunctions may have different signs. Try both and assume that the correct
    # error computation is the one for which the error is minimum
    err_1_plus = u_fun_1_b + u_fun_1_m
    err_2_plus = u_fun_2_b + u_fun_2_m
    err_p_plus = p_fun_b + p_fun_m
    err_1_minus = u_fun_1_b - u_fun_1_m
    err_2_minus = u_fun_2_b - u_fun_2_m
    err_p_minus = p_fun_b - p_fun_m
    err_1_plus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(err_1_plus), ufl.grad(err_1_plus)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    err_2_plus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(err_2_plus), ufl.grad(err_2_plus)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    err_p_plus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(err_p_plus * err_p_plus * ufl.dx), op=mpi4py.MPI.SUM))
    err_1_minus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(err_1_minus), ufl.grad(err_1_minus)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    err_2_minus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(err_2_minus), ufl.grad(err_2_minus)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    err_p_minus_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(err_p_minus * err_p_minus * ufl.dx), op=mpi4py.MPI.SUM))
    u_fun_1_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(u_fun_1_m), ufl.grad(u_fun_1_m)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    u_fun_2_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(ufl.inner(ufl.grad(u_fun_2_m), ufl.grad(u_fun_2_m)) * ufl.dx),
        op=mpi4py.MPI.SUM))
    p_fun_norm = np.sqrt(mesh.comm.allreduce(
        dolfinx.fem.assemble_scalar(p_fun_m * p_fun_m * ufl.dx), op=mpi4py.MPI.SUM))

    def select_error(
        err_plus: dolfinx.fem.Function, err_plus_norm: np.float64,
        err_minus: dolfinx.fem.Function, err_minus_norm: np.float64,
        vec_norm: np.float64, component_name: str
    ) -> None:
        """Pick a +/- sign in the linear combination which defines the error."""
        ratio_plus = err_plus_norm / vec_norm
        ratio_minus = err_minus_norm / vec_norm
        if ratio_minus < ratio_plus:
            print("Relative error for ", component_name, "component of eigenvector equal to",
                  ratio_minus, "(the one with opposite sign was", ratio_plus, ")")
            assert np.isclose(ratio_minus, 0., atol=1.e-6)
        else:
            print("Relative error for", component_name, "component of eigenvector equal to",
                  ratio_plus, "(the one with opposite sign was", ratio_minus, ")")
            assert np.isclose(ratio_plus, 0., atol=1.e-6)

    select_error(err_1_plus, err_1_plus_norm, err_1_minus, err_1_minus_norm, u_fun_1_norm, "velocity 1")
    select_error(err_2_plus, err_2_plus_norm, err_2_minus, err_2_minus_norm, u_fun_2_norm, "velocity 2")
    select_error(err_p_plus, err_p_plus_norm, err_p_minus, err_p_minus_norm, p_fun_norm, "pressure")

In [ ]:
run_error(eig_m, eig_b, u_fun_1_m, u_fun_1_b, u_fun_2_m, u_fun_2_b, p_fun_m, p_fun_b)